In [1]:
from sklearn import datasets
import numpy as np

digits = datasets.load_digits()
X = digits.data
y = digits.target

#### 测试train_test_split

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 666)

In [3]:
from sklearn.neighbors import KNeighborsClassifier
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11): #相邻元素个数
    for p in range(1, 5): #计算距离的参数 p
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        knn_clf.fit(X_train, y_train)
        score = knn_clf.score(X_test, y_test)
        if score > best_score:
            best_score, best_p, best_k =score, p, k

print("best_score = ", best_score)
print("best_p = ", best_p)
print("best_k = ", best_k)

best_score =  0.9866666666666667
best_p =  2
best_k =  5


#### 使用交叉验证进行参数调整

In [4]:
from sklearn.model_selection import cross_val_score #传入一个算法、X_train、y_train  传出k个模型的准确率
knn_clf = KNeighborsClassifier()
cross_val_score(knn_clf, X_train, y_train) #默认分成三份

f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.98896247, 0.98210291, 0.98210291])

In [5]:
best_score, best_p, best_k = 0, 0, 0
for k in range(2, 11): #相邻元素个数
    for p in range(1, 5): #计算距离的参数 p
        knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=k, p=p)
        scores = cross_val_score(knn_clf, X_train, y_train)
        score = np.mean(scores)
        if score > best_score:
            best_score, best_p, best_k =score, p, k

print("best_score = ", best_score)
print("best_p = ", best_p)
print("best_k = ", best_k)

f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
f:\python\lib\site-packages\sklearn\model_select

f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


best_score =  0.9866166891368011
best_p =  3
best_k =  5


In [6]:
best_knn_clf = KNeighborsClassifier(weights='distance', n_neighbors=2, p=2) #使用得到的最佳参数
best_knn_clf.fit(X_train, y_train)
best_knn_clf.score(X_test, y_test)

0.9822222222222222

#### 回顾网格搜索

In [7]:
from sklearn.model_selection import GridSearchCV #这里的cv就是交叉验证的意思

param_grid = [
    {
        'weights' : ['distance'],
        'n_neighbors' : [i for i in range(2, 11)],
        'p' : [i for i in range(1, 6)]
    }
]
grid_search = GridSearchCV(knn_clf, param_grid, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits

f:\python\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Parallel(n_jobs=1)]: Done 135 out of 135 | elapsed:  1.3min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=10, p=4,
                                            weights='distance'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [8]:
grid_search.best_score_

0.9866369710467706

In [9]:
grid_search.best_params_ #最佳参数

{'n_neighbors': 5, 'p': 3, 'weights': 'distance'}

In [10]:
best_knn_clf = grid_search.best_estimator_ #最佳分类器
best_knn_clf.score(X_test, y_test)

0.9822222222222222

In [11]:
cross_val_score(knn_clf, X_train, y_train, cv=5) # cv=5分成五份

array([0.99270073, 1.        , 0.98134328, 0.97752809, 0.97368421])

In [12]:
GridSearchCV(knn_clf, param_grid, verbose=1, cv=5)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=10, p=4,
                                            weights='distance'),
             iid='warn', n_jobs=None,
             param_grid=[{'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                          'p': [1, 2, 3, 4, 5], 'weights': ['distance']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)